In [1]:
!where python

C:\Users\rodes\.conda\envs\plant_env\python.exe
C:\Users\rodes\AppData\Local\Microsoft\WindowsApps\python.exe


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print(tf.__version__)  # Should print 2.16.1


2.20.0


In [6]:
!pip install scikit-learn


   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   -------- ------------------------------- 1.8/8.9 MB 10.1 MB/s eta 0:00:01
   --------------- ------------------------ 3.4/8.9 MB 8.8 MB/s eta 0:00:01
   ----------------- ---------------------- 3.9/8.9 MB 6.3 MB/s eta 0:00:01
   --------------------- ------------------ 4.7/8.9 MB 5.7 MB/s eta 0:00:01
   ------------------------- -------------- 5.8/8.9 MB 5.3 MB/s eta 0:00:01
   ---------------------------------- ----- 7.6/8.9 MB 5.9 MB/s eta 0:00:01
   -------------------------------------- - 8.7/8.9 MB 6.1 MB/s eta 0:00:01
   ---------------------------------------- 8.9/8.9 MB 5.7 MB/s  0:00:01
   ---------------------------------------- 0.0/41.3 MB ? eta -:--:--
   - -------------------------------------- 1.3/41.3 MB 8.4 MB/s eta 0:00:05
   -- ------------------------------------- 2.6/41.3 MB 6.3 MB/s eta 0:00:07
   ---- ----------------------------------- 4.2/41.3 MB 6.8 MB/s eta 0:00:06
   ----- ---------------

In [3]:
import os

print(os.listdir("data"))


['PlantVillage', 'plant_disease_dataset']


In [5]:
import shutil

shutil.unpack_archive("plant_disease.zip", "data/")
print("Extraction done!")



Extraction done!


In [6]:
import os
import shutil

base_dir = "data/plant_disease_dataset"
train_dir = os.path.join(base_dir, "train")
val_dir   = os.path.join(base_dir, "val")

# Remove old train/val folders if they exist
shutil.rmtree(train_dir, ignore_errors=True)
shutil.rmtree(val_dir, ignore_errors=True)

# Create fresh train/val folders
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)


In [7]:
import os
import shutil
import random

# Paths
base_dir = "data/plant_disease_dataset"
source_dir = "data/PlantVillage"  # contains all class subfolders
train_dir = os.path.join(base_dir, "train")
val_dir   = os.path.join(base_dir, "val")

split_ratio = 0.8  # 80% train, 20% val

# Loop through each class folder in source_dir
for class_name in os.listdir(source_dir):
    class_path = os.path.join(source_dir, class_name)
    if os.path.isdir(class_path):
        # Create class subfolders in train and val
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)
        
        # List and shuffle images
        images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
        random.shuffle(images)
        
        split_index = int(len(images) * split_ratio)
        train_images = images[:split_index]
        val_images = images[split_index:]
        
        # Copy images
        for img in train_images:
            shutil.copy(os.path.join(class_path, img), os.path.join(train_dir, class_name, img))
        for img in val_images:
            shutil.copy(os.path.join(class_path, img), os.path.join(val_dir, class_name, img))

print("Images successfully copied to train and val folders!")


Images successfully copied to train and val folders!


In [8]:
import os

train_dir = "data/plant_disease_dataset/train"
val_dir = "data/plant_disease_dataset/val"

# Get list of class folders
train_classes = set(os.listdir(train_dir))
val_classes = set(os.listdir(val_dir))

# Check if class folders match
print("Classes in train but not in val:", train_classes - val_classes)
print("Classes in val but not in train:", val_classes - train_classes)

# Now check images for each class
for cls in train_classes & val_classes:  # common classes
    train_imgs = set(os.listdir(os.path.join(train_dir, cls)))
    val_imgs = set(os.listdir(os.path.join(val_dir, cls)))
    
    common_imgs = train_imgs & val_imgs
    if common_imgs:
        print(f"Some images are in both train and val for class '{cls}': {list(common_imgs)[:5]} ...")


Classes in train but not in val: set()
Classes in val but not in train: set()


In [9]:
import os

train_dir = "data/plant_disease_dataset/train"
val_dir = "data/plant_disease_dataset/val"

# Get list of class folders
train_classes = set(os.listdir(train_dir))
val_classes = set(os.listdir(val_dir))

# Check if class folders match
print("Classes in train but not in val:", train_classes - val_classes)
print("Classes in val but not in train:", val_classes - train_classes)

# Now check images for each class
for cls in train_classes & val_classes:  # common classes
    train_imgs = set(os.listdir(os.path.join(train_dir, cls)))
    val_imgs = set(os.listdir(os.path.join(val_dir, cls)))
    
    common_imgs = train_imgs & val_imgs
    if common_imgs:
        print(f" Some images are in both train and val for class '{cls}': {list(common_imgs)[:5]} ...")


Classes in train but not in val: set()
Classes in val but not in train: set()


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen = ImageDataGenerator(rescale=1./255)
val_gen   = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory("data/plant_disease_dataset/train", target_size=(224,224), batch_size=32)
val_data   = val_gen.flow_from_directory("data/plant_disease_dataset/val", target_size=(224,224), batch_size=32)
print("Done")

Found 16504 images belonging to 16 classes.
Found 4134 images belonging to 16 classes.
Done


In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


In [12]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.2
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    "data/plant_disease_dataset/train",
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    "data/plant_disease_dataset/val",
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical'
)

num_classes = len(train_generator.class_indices)
print("Number of classes:", num_classes)


Found 16504 images belonging to 16 classes.
Found 4134 images belonging to 16 classes.
Number of classes: 16


In [21]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224,224,3))
base_model.trainable = True  # Freeze base layers


x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

In [23]:
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [27]:
epochs = 15
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)



Epoch 1/15
 45/516 ━━━━━━━━━━━━━━━━━━━━ 44:04 6s/step - accuracy: 0.9220 - loss: 0.2488

KeyboardInterrupt: 

In [27]:
model.save("plant_disease_model.keras")


In [29]:
pip install streamlit
